In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
pd.options.display.max_columns=100

In [208]:
sim_output="../output/SF_TourPurposeChoiceModel/calibration-trial-17__bce"

In [3]:
taz_data=gp.read_file('../../scenarios/convert/SF-CHAMP Outputs/TAZ2454_clean.shp')

In [209]:
input_plans=pd.read_csv(sim_output+'/plans.csv.gz')
output_plans=pd.read_csv(sim_output+'/ITERS/it.10/10.plans.csv.gz')
agent_attribs=pd.read_csv('../test/input/sf-test-scenario/Inputs/Test/agent_attribs.csv.gz')

In [13]:
output_plans[(output_plans['personId']=='383-4')&(output_plans['planSelected']==True)]

,personId,planIndex,planScore,planSelected,planElementType,planElementIndex,activityType,activityLocationX,activityLocationY,activityEndTime,legMode,legDepartureTime,legTravelTime,legRouteType,legRouteStartLink,legRouteEndLink,legRouteTravelTime,legRouteDistance,legRouteLinks
3072,383-4,2,67.556392,True,beam.utils.scenario.PlanElement$Activity$@6ae4...,0,home,547413.765429,4.173690e+06,29820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3073,383-4,2,67.556392,True,beam.utils.scenario.PlanElement$Leg$@327d5b74,1,NaN,NaN,NaN,NaN,walk,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
3074,383-4,2,67.556392,True,beam.utils.scenario.PlanElement$Activity$@6ae4...,2,shopping,549049.951755,4.174262e+06,33180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3075,383-4,2,67.556392,True,beam.utils.scenario.PlanElement$Leg$@327d5b74,3,NaN,NaN,NaN,NaN,walk,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
3076,383-4,2,67.556392,True,beam.utils.scenario.PlanElement$Activity$@6ae4...,4,school,549118.294737,4.175055e+06,57300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3077,383-4,2,67.556392,True,beam.utils.scenario.PlanElement$Leg$@327d5b74,5,NaN,NaN,NaN,NaN,walk_transit,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
3078,383-4,2,67.556392,True,beam.utils.scenario.PlanElement$Activity$@6ae4...,6,home,547413.765429,4.173690e+06,10920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
output_events=pd.read_csv(sim_output+'/ITERS/it.10/10.events.csv.gz')

D:\Programs\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning: Columns (0,1,4,5,7,11,12,22,26,33,36,40,43,46,47,50,52,53,54,56,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [211]:
input_events=pd.read_csv(sim_output+'/ITERS/it.0/0.events.csv.gz')

In [9]:
def county(x):
    if x<1191:
        return "San Francisco"
    elif x<1347:
        return "San Mateo"
    elif x<1715:
        return "Santa Clara"
    elif x<2040:
        return "Almeda"
    elif x<2211:
        return "Contra Costa"
    elif x<2291:
        return "Solano"
    elif x<2318:
        return "Napa"
    elif x<2404:
        return 'Sonoma'
    else:
        return "Marin"
    
def asc_var(x):
    
    if x=='auto_deficient':
        return 'ascAutoDeficient'
    elif x=='auto_sufficient':
        return 'ascAutoSufficient'
    else:
        return 'ascNoAuto'
    

def taz_cleanup(taz_df):
    
    taz_df['County']=taz_df['TAZ'].apply(county)
    taz_df['District']=np.where(taz_df['County']=='San Francisco',
                               taz_df['DIST20'],
                               taz_df['County'])
    districts=taz_df[['District','geometry']].dissolve('District')
    
    districts=districts.to_crs('EPSG:26910')
    
    districts=districts.reset_index()
    return districts

def plans_cleanup(plans,districts,agent_attribs):
    
    plans=plans[plans['planSelected']==True]
    plans['Mode']=plans['legMode'].shift(-1)
    plans['maxPlanElementIndex']=plans.groupby('personId')['planElementIndex'].transform(max)
    plans=plans[plans['activityType'].notnull()]
    
    plans['origin_activityType']=plans['activityType']
    plans['tourPurpose']=plans['activityType'].shift(-1)
    
    plans['geometry']=gp.points_from_xy(plans['activityLocationX'],plans['activityLocationY'],crs='EPSG:26910')
    
    plans_gdf=gp.GeoDataFrame(plans,geometry='geometry')
    
    plans_district=gp.sjoin(plans_gdf,districts,how='left',predicate='within')
    
    plans_district['origin_district']=plans_district['District']
    plans_district['destination_district']=plans_district['origin_district'].shift(-1)
    
    plans_district=plans_district[plans_district['planElementIndex']!=plans_district['maxPlanElementIndex']]
    
    plans_df=pd.merge(plans_district,agent_attribs,left_on=['personId'],right_on=['person_id'],how='left')
    
#     plans_district['startTime']=plans_district['activityEndTime']
#     plans_district['startTime']=plans_district['activityEndTime']

    return plans_df[['personId','hhno',  'pgend', 'pagey', 'hhincome', 'hhsize','vehicles',
                     'planIndex', 'planScore', 'planSelected', 'planElementType',
                     'planElementIndex', 'Mode',
                     'maxPlanElementIndex', 'origin_activityType', 'tourPurpose', 'geometry',
                     'index_right', 'District', 'origin_district', 'destination_district']]

def events_cleanup(events,districts,agent_attribs):
    
    _df=events[(events['type'].isin(['ModeChoice','TripArrivalEvent']))][['person','type','time','locationX','locationY','mode','tourIndex','tourPurpose',
                             'activityIndex','actType','currentTourMode',
                            'tracingData','availableAlternatives','vehicleOwnership','length',
                            ]].sort_values(by=['person','time'])

    for col in ['locationX','locationY','activityIndex','actType']:
        _df[col]=_df[col].shift(-1)

    _df=_df[_df['type']=='ModeChoice']
    _df['geometry']=gp.points_from_xy(_df['locationX'],_df['locationY'],crs='EPSG:26910')
    
    events_gdf=gp.GeoDataFrame(_df,geometry='geometry')
    
    events_district=gp.sjoin(events_gdf,districts,how='left',predicate='within')
    
    
    events_df=pd.merge(events_district,agent_attribs,left_on=['person'],right_on=['person_id'],how='left')
    
    events_df['variable']=events_df['vehicleOwnership'].apply(asc_var)

    return events_df
    

In [10]:
%%time
districts=taz_cleanup(taz_data)

CPU times: total: 2.08 s
Wall time: 2.08 s


In [212]:
%%time
output_events_df=events_cleanup(output_events,districts,agent_attribs)

CPU times: total: 6.17 s
Wall time: 6.16 s


In [104]:
import os

In [215]:
directory=os.mkdir(sim_output+'/Debug')

In [217]:
output_mode_count=output_events_df['mode'].value_counts().to_frame().reset_index()
output_mode_count=output_mode_count.rename(columns={'index':'Mode','mode':'Total Count Iteration 20'})
input_mode_count=input_events_df['mode'].value_counts().to_frame().reset_index()
input_mode_count=input_mode_count.rename(columns={'index':'Mode','mode':'Total Count Iteration 0'})
mode_count_comp=pd.merge(output_mode_count,input_mode_count,on=['Mode'],how='outer')
mode_count_comp.to_csv(sim_output+'/Debug/total_mode_comparison calibration-trial-17__bce.csv')

In [214]:
output_events_df['mode'].value_counts()

car                   14561
walk                  11278
walk_transit           6059
hov2                   1834
hov3_teleportation     1704
hov2_teleportation     1517
hov3                    956
ride_hail               839
bike_transit            486
bike                    273
ride_hail_pooled         43
ride_hail_transit        24
drive_transit            11
Name: mode, dtype: int64

In [175]:
input_events_df['mode'].value_counts()

car                   12874
walk                  10155
walk_transit           6271
hov2                   3120
hov3_teleportation     2376
hov2_teleportation     2244
hov3                   1730
ride_hail               572
bike                    222
bike_transit              2
Name: mode, dtype: int64

In [213]:
%%time
input_events_df=events_cleanup(input_events,districts,agent_attribs)

CPU times: total: 7.3 s
Wall time: 7.33 s


In [193]:
mc_params=pd.read_csv('../test/input/sf-test-scenario/Inputs/ActivitySimMC-TSCORE-trial-16.csv')
mc_params

,model,tourType,variable,alternative,units,latentClass,value
0,modeChoice,Person,age010,bike,util,work,0.000
1,modeChoice,Person,age010,bike,util,univ,0.000
2,modeChoice,Person,age010,bike,util,social,0.000
3,modeChoice,Person,age010,bike,util,shopping,0.000
4,modeChoice,Person,age010,bike,util,school,0.000
...,...,...,...,...,...,...,...
2986,modeChoice,Path,walkTime,walk_transit,util/min,othmaint,-0.036
2987,modeChoice,Path,walkTime,walk_transit,util/min,othdiscr,-0.036
2988,modeChoice,Path,walkTime,walk_transit,util/min,escort,-0.036
2989,modeChoice,Path,walkTime,walk_transit,util/min,eatout,-0.036


In [206]:
output_modes_list=output_events_df.groupby(['tourPurpose','variable','mode']).agg(BEAM_Count=('mode','count')).reset_index()
input_modes_list=input_events_df.groupby(['tourPurpose','variable','mode']).agg(SF_CHAMP_Count=('mode','count')).reset_index()


comparison=pd.merge(output_modes_list,input_modes_list,how='left',
                   on=['tourPurpose','variable','mode'])

comparison['SF_CHAMP_Count']=comparison['SF_CHAMP_Count'].fillna(1)
comparison['BEAM_Count']=comparison['BEAM_Count'].fillna(1)

#caar and shared ride, multiply by 2
comparison['Change_in_ASC']=2*np.log(comparison['SF_CHAMP_Count']/comparison['BEAM_Count'])

beam_hov_count=comparison[comparison['mode'].isin(['hov2','hov2_teleportation','hov3','hov3_teleportation'])]['BEAM_Count'].sum()
sfchamp_hov_count=comparison[comparison['mode'].isin(['hov2','hov2_teleportation','hov3','hov3_teleportation'])]['SF_CHAMP_Count'].sum()

change_in_hov_asc=2*np.log(sfchamp_hov_count/beam_hov_count)



comparison['Change_in_ASC']=np.where(comparison['mode'].isin(['hov2','hov2_teleportation','hov3','hov3_teleportation']),
                                    change_in_hov_asc,
                                    comparison['Change_in_ASC'])

comparison['Change_in_ASC']=np.where(comparison['mode'].isin(['car']),
                                    2*comparison['Change_in_ASC'],
                                    comparison['Change_in_ASC'])

comparison

,tourPurpose,variable,mode,BEAM_Count,SF_CHAMP_Count,Change_in_ASC
0,eatout,ascAutoDeficient,bike,1,1.0,0.000000
1,eatout,ascAutoDeficient,bike_transit,8,1.0,-4.158883
2,eatout,ascAutoDeficient,car,346,322.0,-0.287549
3,eatout,ascAutoDeficient,hov2,56,150.0,0.224591
4,eatout,ascAutoDeficient,hov2_teleportation,61,74.0,0.224591
...,...,...,...,...,...,...
210,work,ascNoAuto,bike_transit,1,1.0,0.000000
211,work,ascNoAuto,hov2_teleportation,22,80.0,0.224591
212,work,ascNoAuto,ride_hail,147,171.0,0.302462
213,work,ascNoAuto,walk,762,760.0,-0.005256


In [153]:
comparison[comparison['mode'].isin(['hov2','hov2_teleportation','hov3','hov3_teleportation'])]

#['BEAM_Count'].sum()

,tourPurpose,variable,mode,BEAM_Count,SF_CHAMP_Count,Change_in_ASC
4,eatout,ascAutoDeficient,hov2,43,150.0,0.506507
5,eatout,ascAutoDeficient,hov2_teleportation,25,74.0,0.506507
6,eatout,ascAutoDeficient,hov3,12,69.0,0.506507
7,eatout,ascAutoDeficient,hov3_teleportation,29,47.0,0.506507
17,eatout,ascAutoSufficient,hov2,19,88.0,0.506507
...,...,...,...,...,...,...
215,work,ascAutoSufficient,hov2,159,148.0,0.506507
216,work,ascAutoSufficient,hov2_teleportation,105,45.0,0.506507
217,work,ascAutoSufficient,hov3,36,49.0,0.506507
218,work,ascAutoSufficient,hov3_teleportation,73,35.0,0.506507


In [207]:
updated_mc=pd.merge(mc_params,comparison,
                   left_on=['latentClass','variable','alternative'],
                   right_on=['tourPurpose','variable','mode'],
                   how='left')
updated_mc['Change_in_ASC']=updated_mc['Change_in_ASC'].fillna(0)

updated_mc['updated_value']=updated_mc['value']+updated_mc['Change_in_ASC']

updated_mc.drop(columns=['value'],inplace=True)
updated_mc=updated_mc.rename(columns={'updated_value':'value'})

updated_mc[['model', 'tourType', 'variable', 'alternative', 'units', 'latentClass',
       'value']].to_csv('../test/input/sf-test-scenario/Inputs/ActivitySimMC-TSCORE-trial-17.csv',index=False)
#updated_mc.to_excel('../test/input/sf-test-scenario/Inputs/ActivitySimMC-TSCORE-trial-8-detailed.xlsx')

In [92]:
input_events_df.groupby(['tourPurpose','mode']).agg({'mode':'count'})

mode
tourPurpose mode                    
eatout      bike                   2
            car                  499
            hov2                 238
            hov2_teleportation   125
            hov3                 122
...                              ...
work        hov3                 463
            hov3_teleportation   276
            ride_hail            194
            walk                3316
            walk_transit        3308

[77 rows x 1 columns]

In [87]:
output_events_df['mode'].value_counts()

walk                  16694
car                   11173
walk_transit           4546
hov2                   1968
hov3_teleportation     1600
hov2_teleportation     1578
hov3                   1056
ride_hail               604
drive_transit           174
ride_hail_transit       110
bike_transit             86
ride_hail_pooled         38
bike                      4
Name: mode, dtype: int64

In [145]:
%%time
input_plans_cleaned=plans_cleanup(input_plans,districts,agent_attribs)
output_plans_cleaned=plans_cleanup(output_plans,districts,agent_attribs)

C:\Users\jawad\AppData\Local\Temp\ipykernel_10760\1549724210.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plans['Mode']=plans['legMode'].shift(-1)
C:\Users\jawad\AppData\Local\Temp\ipykernel_10760\1549724210.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plans['maxPlanElementIndex']=plans.groupby('personId')['planElementIndex'].transform(max)


CPU times: total: 21.4 s
Wall time: 21.5 s


In [124]:
events=pd.read_csv('../output/SF_TourPurposeChoiceModel/sf-tscore-all-trips-mc-calibration-trial-8__bgb/ITERS/it.10/10.events.csv.gz')

D:\Programs\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning: Columns (1,5,8,11,14,22,25,35,38,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [126]:
pd.options.display.max_columns=100
events.type.unique()

array(['PersonEntersVehicle', 'ModeChoice', 'actend', 'PathTraversal',
       'ReserveRideHail', 'LeavingParkingEvent', 'ParkingEvent',
       'PersonLeavesVehicle', 'actstart', 'Replanning'], dtype=object)

In [138]:
events[(events['type']=='ModeChoice')].head(50)

,person,reason,time,type,vehicle,vehicleType,shiftStatus,duration,pricingModel,locationX,parkingZoneId,chargingPointType,parkingTaz,fuel,parkingType,locationY,price,primaryFuelLevel,secondaryFuelLevel,link,facility,actType,driver,currentTourMode,primaryFuel,links,numPassengers,departureTime,tollPaid,fromStopIndex,startY,startX,capacity,endX,seatingCapacity,secondaryFuelType,toStopIndex,arrivalTime,primaryFuelType,endY,secondaryFuel,linkTravelTime,riders,mode,length,cost,income,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,vehicleOwnership,tourIndex,tourPurpose,departTime,score
6935,124278-1,NaN,0.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,4337.136000,NaN,0.0,NaN,WALK,25895.0,False,auto_deficient,0.0,social,NaN,NaN
30757,282949-2,NaN,0.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,1399.491000,NaN,0.0,NaN,NaN,22849.0,False,auto_deficient,0.0,othmaint,NaN,NaN
30758,182303-1,NaN,0.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,1513.001000,NaN,0.0,NaN,WALK,24666.0,False,auto_deficient,0.0,eatout,NaN,NaN
30759,273727-3,NaN,0.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hov3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,2692.369000,NaN,0.0,NaN,NaN,37104.0,False,auto_deficient,1.0,social,NaN,NaN
30760,136947-1,NaN,0.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,1524.761000,NaN,0.0,NaN,WALK,23947.0,False,no_auto,0.0,social,NaN,NaN
30761,128177-1,NaN,0.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,7414.017000,NaN,0.0,NaN,WALK,25581.0,False,no_auto,0.0,eatout,NaN,NaN
30762,169180-2,NaN,0.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,2076.124000,NaN,0.0,NaN,WALK,39714.0,False,auto_deficient,0.0,work,NaN,NaN
30763,290899-1,NaN,0.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,4999.517000,NaN,0.0,NaN,NaN,29721.0,False,auto_deficient,0.0,work,NaN,NaN
30764,314164-2,NaN,0.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,610.208000,NaN,0.0,NaN,WALK,40262.0,False,auto_sufficient,0.0,social,NaN,NaN
30765,2764878-1,NaN,0.0,ModeChoice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk,1858.364000,NaN,0.0,NaN,WALK,13347.0,False,no_auto,0.0,social,NaN,NaN


In [143]:
output_plans.head(30)

,personId,planIndex,planScore,planSelected,planElementType,planElementIndex,activityType,activityLocationX,activityLocationY,activityEndTime,legMode,legDepartureTime,legTravelTime,legRouteType,legRouteStartLink,legRouteEndLink,legRouteTravelTime,legRouteDistance,legRouteLinks
0,63-2,0,231.117214,False,beam.utils.scenario.PlanElement$Activity$@423763f,0,home,551662.441744,4.173739e+06,31800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63-2,0,231.117214,False,beam.utils.scenario.PlanElement$Leg$@2486eb62,1,NaN,NaN,NaN,NaN,hov3_teleportation,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
2,63-2,0,231.117214,False,beam.utils.scenario.PlanElement$Activity$@423763f,2,othmaint,549567.289288,4.175127e+06,41520.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,63-2,0,231.117214,False,beam.utils.scenario.PlanElement$Leg$@2486eb62,3,NaN,NaN,NaN,NaN,ride_hail,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
4,63-2,0,231.117214,False,beam.utils.scenario.PlanElement$Activity$@423763f,4,home,551662.441744,4.173739e+06,49440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,63-2,0,231.117214,False,beam.utils.scenario.PlanElement$Leg$@2486eb62,5,NaN,NaN,NaN,NaN,walk,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
6,63-2,0,231.117214,False,beam.utils.scenario.PlanElement$Activity$@423763f,6,othmaint,552229.026758,4.175728e+06,51720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,63-2,0,231.117214,False,beam.utils.scenario.PlanElement$Leg$@2486eb62,7,NaN,NaN,NaN,NaN,walk,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
8,63-2,0,231.117214,False,beam.utils.scenario.PlanElement$Activity$@423763f,8,home,551662.441744,4.173739e+06,10920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,63-2,1,231.117214,False,beam.utils.scenario.PlanElement$Activity$@423763f,0,home,551662.441744,4.173739e+06,31800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
output_plans_cleaned[output_plans_cleaned['Mode'].isnull()].to_csv('Temp/null_modes.csv')

In [67]:
mc_params=pd.read_csv('../test/input/sf-test-scenario/Inputs/ActivitySimMC-TSCORE-trial-5.csv')
mc_params.head()

,model,tourType,variable,alternative,units,latentClass,value
0,modeChoice,Person,cost,walk_transit,util/min,work,-1.0
1,modeChoice,Person,cost,walk,util/min,work,-1.0
2,modeChoice,Person,cost,ride_hail,util/min,work,-1.0
3,modeChoice,Person,cost,hov3_teleportation,util/min,work,-1.0
4,modeChoice,Person,cost,hov3,util/min,work,-1.0


In [63]:
mc_params.variable.unique()

array(['cost', 'ascNoAuto', 'ascAutoSufficient', 'ascAutoDeficient',
       'age1619', 'age010', 'waitTime', 'vehicleTime', 'transfer',
       'shortWalkDist', 'shortBikeDist', 'originFarFromTransit',
       'originCloseToTransit', 'longWalkDist', 'longBikeDist',
       'egressTime', 'destFarFromTransit', 'destCloseToTransit',
       'originZDI', 'destZDI', 'CBD', 'walkTime', 'age16P', 'hhSize1',
       'hhSize2', 'shortDrive'], dtype=object)

In [146]:
df1=input_plans_cleaned.groupby(['tourPurpose','vehicles','Mode']).agg({'Mode':'count'}).rename(columns={'Mode':'SF CHAMP Count'}).reset_index()

output_plans_cleaned['Mode']=np.where(output_plans_cleaned['Mode'].isnull(),
                                     'Not Assigned',
                                     output_plans_cleaned['Mode'])
df2=output_plans_cleaned.groupby(['tourPurpose','vehicles','Mode']).agg({'Mode':'count'}).rename(columns={'Mode':'BEAM Count'}).reset_index()

df3=pd.merge(df1,df2,on=['tourPurpose','vehicles','Mode'],how='outer')

df3['Change_in_ASC']=np.log(df3['SF CHAMP Count']/df3['BEAM Count'])

df3.head(10)
#.to_csv('Temp/mode_counts.csv')

,tourPurpose,vehicles,Mode,SF CHAMP Count,BEAM Count,Change_in_ASC
0,eatout,auto_deficient,bike,27.0,34.0,-0.230524
1,eatout,auto_deficient,car,270.0,26.0,2.340325
2,eatout,auto_deficient,hov2,135.0,19.0,1.960836
3,eatout,auto_deficient,hov2_teleportation,66.0,7.0,2.243745
4,eatout,auto_deficient,hov3,51.0,14.0,1.292768
5,eatout,auto_deficient,hov3_teleportation,55.0,23.0,0.871839
6,eatout,auto_deficient,ride_hail,76.0,94.0,-0.212561
7,eatout,auto_deficient,walk,367.0,563.0,-0.427918
8,eatout,auto_deficient,walk_transit,89.0,127.0,-0.355551
9,eatout,auto_sufficient,bike,13.0,6.0,0.773190


In [65]:
df3.vehicles.unique()

array(['auto_deficient', 'auto_sufficient', 'no_auto'], dtype=object)

In [147]:
def asc_var(x):
    
    if x=='auto_deficient':
        return 'ascAutoDeficient'
    elif x=='auto_sufficient':
        return 'ascAutoSufficient'
    else:
        return 'ascNoAuto'
    
df3['variable']=df3['vehicles'].apply(asc_var)
df3.head()

,tourPurpose,vehicles,Mode,SF CHAMP Count,BEAM Count,Change_in_ASC,variable
0,eatout,auto_deficient,bike,27.0,34.0,-0.230524,ascAutoDeficient
1,eatout,auto_deficient,car,270.0,26.0,2.340325,ascAutoDeficient
2,eatout,auto_deficient,hov2,135.0,19.0,1.960836,ascAutoDeficient
3,eatout,auto_deficient,hov2_teleportation,66.0,7.0,2.243745,ascAutoDeficient
4,eatout,auto_deficient,hov3,51.0,14.0,1.292768,ascAutoDeficient


In [59]:
df3.to_csv('Temp/trial-8-mode_counts.csv')

In [68]:
updated_mc_param=pd.merge(mc_params,df3,
                          left_on=['alternative','latentClass','variable'],
                         right_on=['Mode','tourPurpose','variable'],
                         how='left')
updated_mc_param['Change_in_ASC']=updated_mc_param['Change_in_ASC'].fillna(0)

updated_mc_param['updated_value']=updated_mc_param['value']+updated_mc_param['Change_in_ASC']

#updated_mc_param.drop(columns=['Mode','tourPurpose','Change_in_ASC','value'],inplace=True)

#updated_mc_param=updated_mc_param.rename(columns={'updated_value':'value'})

updated_mc_param
#.to_csv('../test/input/sf-test-scenario/Inputs/MC_Parameters_Trial_4_Detailed.csv',index=False)

,model,tourType,variable,alternative,units,latentClass,value,tourPurpose,vehicles,Mode,SF CHAMP Count,BEAM Count,Change_in_ASC,updated_value
0,modeChoice,Person,cost,walk_transit,util/min,work,-1.0,NaN,NaN,NaN,NaN,NaN,0.0,-1.0
1,modeChoice,Person,cost,walk,util/min,work,-1.0,NaN,NaN,NaN,NaN,NaN,0.0,-1.0
2,modeChoice,Person,cost,ride_hail,util/min,work,-1.0,NaN,NaN,NaN,NaN,NaN,0.0,-1.0
3,modeChoice,Person,cost,hov3_teleportation,util/min,work,-1.0,NaN,NaN,NaN,NaN,NaN,0.0,-1.0
4,modeChoice,Person,cost,hov3,util/min,work,-1.0,NaN,NaN,NaN,NaN,NaN,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2986,modeChoice,Path,walkTime,ride_hail_transit,util/min,atwork,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2987,modeChoice,Person,age16P,ride_hail_transit,util,atwork,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2988,modeChoice,Person,hhSize1,ride_hail_transit,util,atwork,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2989,modeChoice,Person,hhSize2,ride_hail_transit,util,atwork,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [44]:
updated_mc_param[updated_mc_param['variable'].isin(['ascAutoDeficient','ascAutoSuficient','ascNoAuto'])]

,model,tourType,variable,alternative,units,latentClass,value,tourPurpose,vehicles,Mode,SF CHAMP Count,BEAM Count,Change_in_ASC,updated_value
10,modeChoice,Person,ascNoAuto,walk_transit,util,work,5.030,work,no_auto,walk_transit,1343.0,431.0,1.136553,6.166553
11,modeChoice,Person,ascNoAuto,walk,util,work,5.725,work,no_auto,walk,999.0,1988.0,-0.688130,5.036870
12,modeChoice,Person,ascNoAuto,ride_hail,util,work,5.030,work,no_auto,ride_hail,662.0,24.0,3.317212,8.347212
13,modeChoice,Person,ascNoAuto,hov3_teleportation,util,work,-0.629,work,no_auto,hov3_teleportation,47.0,5.0,2.240710,1.611710
14,modeChoice,Person,ascNoAuto,hov3,util,work,-0.629,NaN,NaN,NaN,NaN,NaN,0.000000,-0.629000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2916,modeChoice,Person,ascAutoDeficient,ride_hail_transit,util,escort,-1.626,NaN,NaN,NaN,NaN,NaN,0.000000,-1.626000
2940,modeChoice,Person,ascNoAuto,ride_hail_transit,util,eatout,-99.000,NaN,NaN,NaN,NaN,NaN,0.000000,-99.000000
2942,modeChoice,Person,ascAutoDeficient,ride_hail_transit,util,eatout,0.482,NaN,NaN,NaN,NaN,NaN,0.000000,0.482000
2966,modeChoice,Person,ascNoAuto,ride_hail_transit,util,atwork,-99.000,NaN,NaN,NaN,NaN,NaN,0.000000,-99.000000


In [97]:
df3.sort_values(by=['tourPurpose'],ascending=False).to_csv('Temp/trial-5-mode_counts.csv',index=False)

In [69]:
updated_mc_param.drop(columns=['tourPurpose', 'vehicles', 'Mode', 'SF CHAMP Count', 'BEAM Count',
       'Change_in_ASC','value'],inplace=True)

updated_mc_param=updated_mc_param.rename(columns={'updated_value':'value'})

updated_mc_param.head()

,model,tourType,variable,alternative,units,latentClass,value
0,modeChoice,Person,cost,walk_transit,util/min,work,-1.0
1,modeChoice,Person,cost,walk,util/min,work,-1.0
2,modeChoice,Person,cost,ride_hail,util/min,work,-1.0
3,modeChoice,Person,cost,hov3_teleportation,util/min,work,-1.0
4,modeChoice,Person,cost,hov3,util/min,work,-1.0


In [70]:
updated_mc_param.to_csv('../test/input/sf-test-scenario/Inputs/ActivitySimMC-TSCORE-trial-6.csv',index=False)

In [56]:
output_plans_cleaned.head(10)

,personId,hhno,pgend,pagey,hhincome,hhsize,vehicles,planIndex,planScore,planSelected,...,planElementIndex,Mode,maxPlanElementIndex,origin_activityType,tourPurpose,geometry,index_right,District,origin_district,destination_district
0,63-2,63,female,32,38016,8.0,auto_sufficient,4,379.596677,True,...,0,Not Assigned,8,home,othmaint,POINT (551662.442 4173738.730),8.0,9,9,10
1,63-2,63,female,32,38016,8.0,auto_sufficient,4,379.596677,True,...,2,Not Assigned,8,othmaint,home,POINT (549567.289 4175127.022),9.0,10,10,9
2,63-2,63,female,32,38016,8.0,auto_sufficient,4,379.596677,True,...,4,Not Assigned,8,home,othmaint,POINT (551662.442 4173738.730),8.0,9,9,9
3,63-2,63,female,32,38016,8.0,auto_sufficient,4,379.596677,True,...,6,Not Assigned,8,othmaint,home,POINT (552229.027 4175727.623),8.0,9,9,9
4,63-3,63,female,30,38016,8.0,auto_sufficient,2,930.963822,True,...,0,Not Assigned,16,home,escort,POINT (551662.442 4173738.730),8.0,9,9,10
5,63-3,63,female,30,38016,8.0,auto_sufficient,2,930.963822,True,...,2,Not Assigned,16,escort,home,POINT (548946.307 4174811.777),9.0,10,10,9
6,63-3,63,female,30,38016,8.0,auto_sufficient,2,930.963822,True,...,4,Not Assigned,16,home,escort,POINT (551662.442 4173738.730),8.0,9,9,10
7,63-3,63,female,30,38016,8.0,auto_sufficient,2,930.963822,True,...,6,Not Assigned,16,escort,home,POINT (549880.859 4174499.154),9.0,10,10,9
8,63-3,63,female,30,38016,8.0,auto_sufficient,2,930.963822,True,...,8,Not Assigned,16,home,shopping,POINT (551662.442 4173738.730),8.0,9,9,1
9,63-3,63,female,30,38016,8.0,auto_sufficient,2,930.963822,True,...,10,Not Assigned,16,shopping,home,POINT (553319.371 4183077.012),0.0,1,1,9


In [148]:
trips_comparison=pd.merge(input_plans_cleaned[['personId', 'pgend', 'pagey', 'hhincome', 'hhsize','vehicles',
                                               'planElementIndex', 'Mode', 'origin_activityType', 'tourPurpose',
                                               'origin_district', 'destination_district']],
                         output_plans_cleaned[['personId', 'pgend', 'pagey', 'hhincome', 'hhsize','vehicles',
                                               'planElementIndex', 'Mode', 'origin_activityType', 'tourPurpose',
                                               'origin_district', 'destination_district']],
                         on=['personId', 'pgend', 'pagey', 'hhincome', 'hhsize','vehicles','planElementIndex'],
                         how='inner',
                         suffixes=('_SFCHAMP','_BEAM'))

trips_comparison.head()

,personId,pgend,pagey,hhincome,hhsize,vehicles,planElementIndex,Mode_SFCHAMP,origin_activityType_SFCHAMP,tourPurpose_SFCHAMP,origin_district_SFCHAMP,destination_district_SFCHAMP,Mode_BEAM,origin_activityType_BEAM,tourPurpose_BEAM,origin_district_BEAM,destination_district_BEAM
0,63-3,female,30,38016,8.0,auto_sufficient,0,hov3_teleportation,home,escort,9,10,walk,home,escort,9,10
1,63-3,female,30,38016,8.0,auto_sufficient,2,hov3,escort,home,10,9,walk,escort,home,10,9
2,63-3,female,30,38016,8.0,auto_sufficient,4,hov2_teleportation,home,escort,9,10,walk,home,escort,9,10
3,63-3,female,30,38016,8.0,auto_sufficient,6,hov3,escort,home,10,9,walk,escort,home,10,9
4,63-3,female,30,38016,8.0,auto_sufficient,8,hov3_teleportation,home,shopping,9,1,walk,home,shopping,9,1


In [149]:
output_plans_cleaned['ext']=output_plans_cleaned['personId'].apply(lambda x: x[0:2])


In [23]:
output_plans[output_plans['personId']=='ix-15241-1']

,personId,planIndex,planScore,planSelected,planElementType,planElementIndex,activityType,activityLocationX,activityLocationY,activityEndTime,legMode,legDepartureTime,legTravelTime,legRouteType,legRouteStartLink,legRouteEndLink,legRouteTravelTime,legRouteDistance,legRouteLinks
62829,ix-15241-1,0,974.74203,True,beam.utils.scenario.PlanElement$Activity$@423763f,0,othmaint,545843.872793,4.168884e+06,2040.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62830,ix-15241-1,0,974.74203,True,beam.utils.scenario.PlanElement$Leg$@2486eb62,1,NaN,NaN,NaN,NaN,car,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
62831,ix-15241-1,0,974.74203,True,beam.utils.scenario.PlanElement$Activity$@423763f,2,othmaint,548800.493770,4.178335e+06,5280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62832,ix-15241-1,0,974.74203,True,beam.utils.scenario.PlanElement$Leg$@2486eb62,3,NaN,NaN,NaN,NaN,car,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
62833,ix-15241-1,0,974.74203,True,beam.utils.scenario.PlanElement$Activity$@423763f,4,work,589193.755587,4.138435e+06,60720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62834,ix-15241-1,0,974.74203,True,beam.utils.scenario.PlanElement$Leg$@2486eb62,5,NaN,NaN,NaN,NaN,car,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
62835,ix-15241-1,0,974.74203,True,beam.utils.scenario.PlanElement$Activity$@423763f,6,social,552566.303270,4.174617e+06,72720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62836,ix-15241-1,0,974.74203,True,beam.utils.scenario.PlanElement$Leg$@2486eb62,7,NaN,NaN,NaN,NaN,car,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN
62837,ix-15241-1,0,974.74203,True,beam.utils.scenario.PlanElement$Activity$@423763f,8,othmaint,551464.066663,4.178593e+06,83520.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62838,ix-15241-1,0,974.74203,True,beam.utils.scenario.PlanElement$Leg$@2486eb62,9,NaN,NaN,NaN,NaN,car,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
output_plans_cleaned[output_plans_cleaned['personId']=='ix-15241-1']

,personId,hhno,pgend,pagey,hhincome,hhsize,vehicles,planIndex,planScore,planSelected,planElementType,planElementIndex,Mode,maxPlanElementIndex,origin_activityType,tourPurpose,geometry,index_right,District,origin_district,destination_district,ext
6366,ix-15241-1,15241,male,31,88073,4.0,auto_deficient,0,974.74203,True,beam.utils.scenario.PlanElement$Activity$@423763f,0,car,10,othmaint,othmaint,POINT (545843.873 4168883.722),17.0,San Mateo,San Mateo,11,ix
6367,ix-15241-1,15241,male,31,88073,4.0,auto_deficient,0,974.74203,True,beam.utils.scenario.PlanElement$Activity$@423763f,2,car,10,othmaint,work,POINT (548800.494 4178335.480),10.0,11,11,Santa Clara,ix
6368,ix-15241-1,15241,male,31,88073,4.0,auto_deficient,0,974.74203,True,beam.utils.scenario.PlanElement$Activity$@423763f,4,car,10,work,social,POINT (589193.756 4138435.306),18.0,Santa Clara,Santa Clara,9,ix
6369,ix-15241-1,15241,male,31,88073,4.0,auto_deficient,0,974.74203,True,beam.utils.scenario.PlanElement$Activity$@423763f,6,car,10,social,othmaint,POINT (552566.303 4174617.020),8.0,9,9,5,ix
6370,ix-15241-1,15241,male,31,88073,4.0,auto_deficient,0,974.74203,True,beam.utils.scenario.PlanElement$Activity$@423763f,8,car,10,othmaint,othmaint,POINT (551464.067 4178593.396),4.0,5,5,San Mateo,ix


In [22]:
output_plans_cleaned[(output_plans_cleaned['ext']=='ix')&(output_plans_cleaned['maxPlanElementIndex']>8)]

,personId,hhno,pgend,pagey,hhincome,hhsize,vehicles,planIndex,planScore,planSelected,planElementType,planElementIndex,Mode,maxPlanElementIndex,origin_activityType,tourPurpose,geometry,index_right,District,origin_district,destination_district,ext
6366,ix-15241-1,15241,male,31,88073,4.0,auto_deficient,0,974.742030,True,beam.utils.scenario.PlanElement$Activity$@423763f,0,car,10,othmaint,othmaint,POINT (545843.873 4168883.722),17.0,San Mateo,San Mateo,11,ix
6367,ix-15241-1,15241,male,31,88073,4.0,auto_deficient,0,974.742030,True,beam.utils.scenario.PlanElement$Activity$@423763f,2,car,10,othmaint,work,POINT (548800.494 4178335.480),10.0,11,11,Santa Clara,ix
6368,ix-15241-1,15241,male,31,88073,4.0,auto_deficient,0,974.742030,True,beam.utils.scenario.PlanElement$Activity$@423763f,4,car,10,work,social,POINT (589193.756 4138435.306),18.0,Santa Clara,Santa Clara,9,ix
6369,ix-15241-1,15241,male,31,88073,4.0,auto_deficient,0,974.742030,True,beam.utils.scenario.PlanElement$Activity$@423763f,6,car,10,social,othmaint,POINT (552566.303 4174617.020),8.0,9,9,5,ix
6370,ix-15241-1,15241,male,31,88073,4.0,auto_deficient,0,974.742030,True,beam.utils.scenario.PlanElement$Activity$@423763f,8,car,10,othmaint,othmaint,POINT (551464.067 4178593.396),4.0,5,5,San Mateo,ix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141087,ix-2738481-3,2738481,male,23,71462,3.0,auto_sufficient,2,100.930584,True,beam.utils.scenario.PlanElement$Activity$@423763f,0,walk,10,escort,home,POINT (551173.851 4183965.182),2.0,3,3,Marin,ix
141088,ix-2738481-3,2738481,male,23,71462,3.0,auto_sufficient,2,100.930584,True,beam.utils.scenario.PlanElement$Activity$@423763f,2,walk,10,home,othmaint,POINT (540910.599 4191706.427),15.0,Marin,Marin,8,ix
141089,ix-2738481-3,2738481,male,23,71462,3.0,auto_sufficient,2,100.930584,True,beam.utils.scenario.PlanElement$Activity$@423763f,4,walk,10,othmaint,home,POINT (547134.008 4181544.012),7.0,8,8,Marin,ix
141090,ix-2738481-3,2738481,male,23,71462,3.0,auto_sufficient,2,100.930584,True,beam.utils.scenario.PlanElement$Activity$@423763f,6,walk,10,home,othmaint,POINT (540910.599 4191706.427),15.0,Marin,Marin,7,ix


In [18]:
pd.options.display.max_columns=50
output_plans_cleaned[output_plans_cleaned['personId']=='ix-131-1']

,personId,hhno,pgend,pagey,hhincome,hhsize,vehicles,planIndex,planScore,planSelected,planElementType,planElementIndex,Mode,maxPlanElementIndex,origin_activityType,tourPurpose,geometry,index_right,District,origin_district,destination_district,ext
41,ix-131-1,131,female,23,29512,3.0,auto_deficient,1,-33.711452,True,beam.utils.scenario.PlanElement$Activity$@423763f,0,hov2_teleportation,4,home,work,POINT (551662.442 4173738.730),8.0,9,9,Santa Clara,ix
42,ix-131-1,131,female,23,29512,3.0,auto_deficient,1,-33.711452,True,beam.utils.scenario.PlanElement$Activity$@423763f,2,hov2_teleportation,4,work,home,POINT (578247.482 4137082.984),18.0,Santa Clara,Santa Clara,9,ix


In [150]:
output_plans_cleaned[output_plans_cleaned['ext']=='ix'].Mode.value_counts()

Not Assigned          1276
hov3_teleportation    1232
car                    957
hov2_teleportation     863
walk_transit           508
hov2                   507
hov3                   353
ride_hail              282
walk                   111
ride_hail_pooled        86
bike_transit            72
Name: Mode, dtype: int64

In [46]:
output_plans_cleaned[output_plans_cleaned['ext']=='ix'].Mode.value_counts()

car                   6318
Not Assigned          6240
hov3_teleportation    3586
hov2_teleportation    2826
hov2                  1795
hov3                  1132
walk                   741
Name: Mode, dtype: int64

In [16]:
output_plans_cleaned[output_plans_cleaned['ext']=='ix'].Mode.value_counts()

car                   5259
walk                  4960
Not Assigned          3857
hov3_teleportation    2874
hov2_teleportation    2710
hov2                  1444
walk_transit           658
hov3                   575
ride_hail              166
bike                    70
bike_transit            39
ride_hail_pooled        26
Name: Mode, dtype: int64

In [151]:
def replanning(df):
    if df['Mode_BEAM']=='Not Assigned':
        return 'Null Mode'
    
    elif df['Mode_BEAM']==df['Mode_SFCHAMP']:
        return 'Mode not replanned'
    
    else:
        return 'Mode Replanned'
    
trips_comparison['Mode_Replanning']=trips_comparison.apply(replanning,axis=1)

In [106]:
not_assigned_persons=output_plans_cleaned[output_plans_cleaned['Mode']=='Not Assigned'].personId.unique()

In [103]:
trips_comparison.Mode_Replanning.value_counts().to_csv('trial-5-mode_replanning-status.csv')

In [152]:
trips_comparison.Mode_Replanning.value_counts()

Mode Replanned        23420
Mode not replanned     8268
Null Mode              7904
Name: Mode_Replanning, dtype: int64

In [63]:
trips_comparison.Mode_Replanning.value_counts(normalize=True)

Mode Replanned        0.507662
Mode not replanned    0.321918
Null Mode             0.170420
Name: Mode_Replanning, dtype: float64

In [105]:
output_plans_cleaned['Mode'].value_counts(dropna=False).to_csv('trial-5-mode-counts.csv')

In [156]:
trips_comparison[trips_comparison['personId']=='ix-9612-5']

,personId,pgend,pagey,hhincome,hhsize,vehicles,planElementIndex,Mode_SFCHAMP,origin_activityType_SFCHAMP,tourPurpose_SFCHAMP,origin_district_SFCHAMP,destination_district_SFCHAMP,Mode_BEAM,origin_activityType_BEAM,tourPurpose_BEAM,origin_district_BEAM,destination_district_BEAM,Mode_Replanning
1411,ix-9612-5,female,18,29185,4.0,auto_deficient,0,car,othdiscr,home,Almeda,9,car,othdiscr,home,Almeda,9,Mode not replanned


In [153]:
output_plans_cleaned['Mode'].value_counts(dropna=False,normalize=True)

walk                  0.405461
Not Assigned          0.199636
walk_transit          0.132476
ride_hail             0.063472
bike                  0.048015
hov3_teleportation    0.035891
car                   0.035386
hov2_teleportation    0.024955
hov2                  0.023313
hov3                  0.017832
bike_transit          0.011391
ride_hail_pooled      0.002172
Name: Mode, dtype: float64

In [65]:
output_plans_cleaned['Mode'].value_counts(dropna=False,normalize=True)
#.to_csv('Temp/Trial-3-MC.csv')

walk                  0.456494
Not Assigned          0.170420
walk_transit          0.118703
car                   0.066493
bike                  0.041301
hov2_teleportation    0.032622
hov2                  0.032017
hov3_teleportation    0.031281
ride_hail             0.031281
hov3                  0.014283
bike_transit          0.004687
other                 0.000229
ride_hail_pooled      0.000187
Name: Mode, dtype: float64

In [45]:
input_plans_cleaned[input_plans_cleaned['index_right'].isnull()]

,personId,hhno,pgend,pagey,hhincome,hhsize,vehicles,planIndex,planScore,planSelected,...,planElementIndex,Mode,maxPlanElementIndex,origin_activityType,tourPurpose,geometry,index_right,District,origin_district,destination_district
3210,ix-8134-3,8134,male,36,87015,6.0,auto_deficient,0,0.0,True,...,2,car,4,work,home,POINT (553795.225 4168899.055),NaN,NaN,NaN,9
3550,8953-2,8953,male,53,36262,3.0,auto_sufficient,0,0.0,True,...,2,walk,8,work,eatout,POINT (545251.626 4173848.876),NaN,NaN,NaN,12
3552,8953-2,8953,male,53,36262,3.0,auto_sufficient,0,0.0,True,...,6,hov2,8,work,home,POINT (545251.626 4173848.876),NaN,NaN,NaN,10
4464,ix-10893-1,10893,female,60,58353,2.0,auto_deficient,0,0.0,True,...,6,car,8,shopping,home,POINT (560723.671 4186039.688),NaN,NaN,NaN,9
5325,ix-12657-1,12657,male,60,3293,2.0,auto_sufficient,0,0.0,True,...,2,car,4,work,home,POINT (553795.225 4168899.055),NaN,NaN,NaN,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140779,ix-2734940-3,2734940,male,27,34323,3.0,auto_deficient,0,0.0,True,...,0,car,4,work,work,POINT (544246.946 4191286.758),NaN,NaN,NaN,San Mateo
141280,ix-2740930-2,2740930,male,37,49262,4.0,auto_deficient,0,0.0,True,...,0,car,4,work,work,POINT (544246.946 4191286.758),NaN,NaN,NaN,12
141458,ix-2743908-1,2743908,female,81,30828,1.0,auto_sufficient,0,0.0,True,...,0,car,4,home,shopping,POINT (530481.606 4195035.742),NaN,NaN,NaN,8
141460,ix-2743956-1,2743956,female,64,68483,1.0,auto_sufficient,0,0.0,True,...,0,car,4,home,work,POINT (530481.606 4195035.742),NaN,NaN,NaN,9
